In [1]:
from tqdm import *

In [2]:
import os
import sys
import csv
import time
import argparse
import fnmatch
import librosa
import pandas as pd

from hparams import HParams as hp
from zipfile import ZipFile
from audio import preprocess
from utils import download_file
from datasets.tsp_speech import TSPSpeech
from datasets.mb_speech import MBSpeech
from datasets.lj_speech import LJSpeech

In [3]:
import sys
import time
import argparse


import numpy as np

import torch
import torch.nn.functional as F

# project imports
from models import Text2Mel
from hparams import HParams as hp
from logger import Logger
from utils import get_last_checkpoint_file_name, load_checkpoint, save_checkpoint
from datasets.data_loader import Text2MelDataLoader

In [4]:
dataset_path = "datasets/mathias_dataset/"
tsp_speech = TSPSpeech([],"mathias_dataset/", "mathias_metadata.csv")

191-0000|Hi, and welcome back to another deep learning lecture.|Hi, and welcome back to another deep learning lecture.

191-0001|Today, we will be talking about THE topic that you have all been waiting for.|Today, we will be talking about THE topic that you have all been waiting for.

191-0002|We are at Hopkins.|We are at Hopkins.

191-0003|No deep learning course, and no course probably in general at Hopkins, can go|No deep learning course, and no course probably in general at Hopkins, can go

191-0004|Without a specialized lecture, at least, on healthcare.|Without a specialized, lecture, at least, on healthcare.

191-0005|And today we will presenting some of the recent successes, some of the challenges, and in general, some thoughts on, the revolution, applications, and challenges of deep learning.|And today we will presenting some of the recent successes, some of the challenges, and in general, some thoughts on, the revolution, applications, and challenges of deep learning.

191-000

ValueError: not enough values to unpack (expected 3, got 1)

In [ ]:
tsp_speech.fnames

In [ ]:
# Do preprocessing
preprocess(dataset_path, tsp_speech)

In [ ]:
train_data_loader = Text2MelDataLoader(text2mel_dataset=TSPSpeech(['texts', 'mels', 'mel_gates']), batch_size=2,
                                       mode='train')

In [ ]:
next(iter(train_data_loader))

In [ ]:
TSPSpeech(['texts', 'mels', 'mel_gates'])

In [ ]:
testing = np.load("datasets/TSP_MD_Speaker/mels/MD19_01.npy")
import matplotlib.pyplot as plt
plt.imshow(testing.T, origin='lower')

In [ ]:
testing = np.load("datasets/LJSpeech-1.1/mels/LJ001-0001.npy")
import matplotlib.pyplot as plt
plt.imshow(testing.T, origin='lower')

In [ ]:
import os
import re
import codecs
import unicodedata
import numpy as np

vocab = "PE abcdefghijklmnopqrstuvwxyz'.?"  # P: Padding, E: EOS.
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = {idx: char for idx, char in enumerate(vocab)}


def text_normalize(text):
    text = ''.join(char for char in unicodedata.normalize('NFD', text)
                   if unicodedata.category(char) != 'Mn')  # Strip accents

    text = text.lower()
    text = re.sub("[^{}]".format(vocab), " ", text)
    text = re.sub("[ ]+", " ", text)
    return text


def read_metadata(metadata_file):
    fnames, text_lengths, texts = [], [], []
    print(fnames)
    transcript = os.path.join(metadata_file)
    lines = codecs.open(transcript, 'r', 'utf-8-sig').readlines()
    for line in lines:
        fname, _, text = line.strip().split("|")
        print(fname)
        fnames.append(fname)
        print(fnames)
        text = text_normalize(text) + "E"  # E: EOS
        text = [char2idx[char] for char in text]
        text_lengths.append(len(text))
        texts.append(np.array(text, np.long))
    print(fnames, text_lengths, texts, sep="\n")
    return fnames, text_lengths, texts


def get_test_data(sentences, max_n):
    normalized_sentences = [text_normalize(line).strip() + "E" for line in sentences]  # text normalization, E: EOS
    texts = np.zeros((len(normalized_sentences), max_n + 1), np.long)
    for i, sent in enumerate(normalized_sentences):
        texts[i, :len(sent)] = [char2idx[char] for char in sent]
    return texts


In [ ]:
os.getcwd()

In [ ]:
temp = read_metadata('datasets/TSP_M_Speakers/TSP-M_metadata.csv')